In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate
# !pip install mutagen
# !pip install srt

In [3]:
from huggingface_hub import login
# load the token from txt file
with open("HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user_7542/.cache/huggingface/token
Login successful


In [4]:
import os
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, RoomSimulator
import srt

NEWDATA = False
ADDTOKENS = True
NIKUD = False # False to remove the nikud
JUST_TEAMIM = False
BASE_CHAR = "@"
NUSACHIM =  ["ashkenazi", "maroko", "yerushalmi", "bavly"] #["ashkenazi", "maroko", "yerushalmi", "bavly"]


FASTTEST = False
BATCH_SIZE = 8

SR = 16000
RANDOM = True 
AUGMENT = False

LR = 1e-5
WARMUP_STEPS = 100
EVAL_STEPS = 100    
SAVE_STEPS = 10000
MAX_STEPS = 20000
DROPOUT = False # False or a number between 0 and 1
WEIGHT_DECAY = False # False or a number

EVALUATE_FIRST_STEP = True # if True, will evaluate the model after the first step

#base model 
BASE_MODEL_VERSIONS = ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3"] # for v3 we need to change the log-mel spectrum
BASE_MODEL_VERSION = BASE_MODEL_VERSIONS[3] # num of model. 0=tiny 1=base... 6=large-v3
# BASE_MODEL_NAME = "openai/whisper-" + BASE_MODEL_VERSION
BASE_MODEL_NAME = "cantillation/Teamim-AllNusah-whisper-medium_Random-True_Mid"
# BASE_MODEL_NAME = "ivrit-ai/whisper-v2-d3-e3" best hebrew model (fine-tuned, large-v2 model)
# other hebrew model:
# "BenShermaister/whisper-medium-he"


RUN_NAME = BASE_MODEL_VERSION + "_Random-" + str(RANDOM) + (("_DropOut-" + str(DROPOUT)) if DROPOUT else "") + (("_WeightDecay-" + str(WEIGHT_DECAY)) if WEIGHT_DECAY else "")  + "_Augmented"*AUGMENT \
                                                                                + "_OriginalData" if NEWDATA else ""   # + "_Warmup_steps-" + str(WARMUP_STEPS) + "_Eval_steps-" + str(EVAL_STEPS) + "_Save_steps-" + str(SAVE_STEPS) + "_Max_steps-" + str(MAX_STEPS) # + "_EvalFirstStep-" + str(EVALUATE_FIRST_STEP)  "_LR-" + str(LR) + 



#the new model - after training 
MODEL_NAME = f"./Teamim-{RUN_NAME}" # because the run name doesn't work, I added it to the model name



In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hebrew", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
if ADDTOKENS:
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim

    processor.tokenizer.add_tokens(new_tokens)
    
    
        
        
    

In [7]:
def remove_nikud(text): #TODO IMPORT FROM THE FILE 
    nikud_list = ["ֱ","ֲ","ֳ","ִ","ֵ","ֶ","ַ","ָ","ׂ","ׁ","ֹ","ּ","ֻ","ְ","ׇ"]
    for nikud in nikud_list:
        text = text.replace(nikud, "")
    return text

def just_teamim(text, base_char = BASE_CHAR):#TODO IMPORT FROM THE FILE 
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    new_text = ""
    for char in text:
        if char in teamim:
            new_text += base_char
            new_text += char
        elif char == " ":
            new_text += " "
    return new_text

In [8]:
# path = "/content/drive/Othercomputers/My Laptop/Project/data/PocketTorah/"
# path = "/content/drive/MyDrive/PocketTorah/"
path = "."

In [9]:
class parashat_hashavua_dataset:
        def __init__(self, new_data, few_data=False, train = True ,validation=False, test=False, num_of_words_in_sample = 15, random = False, prob_for_num_of_parts=[], nusachim=["ashkenazi"], augment=False, load_cantillationless_data = False):
                self.data = []
                self.few_data = few_data
                self.load_data(new_data, train, validation, test, nusachim=nusachim, load_cantillationless_data=load_cantillationless_data)
                if JUST_TEAMIM:
                        self.data['text'] = self.data['text'].apply(just_teamim)
                elif not NIKUD:
                        self.data['text'] = self.data['text'].apply(remove_nikud)
                self.data = self.data[self.data['text'] != ""] # remove empty texts (and their audio)
                self.num_of_words_in_sample = num_of_words_in_sample
                self.random = random
                self.start = 0
                self.is_eval_set = validation or test
                self.prob_for_num_of_parts = prob_for_num_of_parts if prob_for_num_of_parts else [1/self.num_of_words_in_sample for i in range(self.num_of_words_in_sample)]
                self.augment = augment
                # prob_for_num_of_parts - the probability to take 1, 2, 3, etc. parts.
                # example of prob_for_num_of_parts: [0.1, 0.2, 0.3, 0.4] means that the probability to take 1 part is 0.1, 2 parts is 0.2, etc.

        def __getitem__(self, index):
                if self.is_eval_set:
                        audio, text_tokens, _ = self.get_sequence_(index*self.num_of_words_in_sample, num_of_words=self.num_of_words_in_sample)
                else:
                        if self.random:
                                # ensure that the sum of probabilities is 1
                                if np.sum(self.prob_for_num_of_parts) != 1:
                                        self.prob_for_num_of_parts = self.prob_for_num_of_parts / np.sum(self.prob_for_num_of_parts)
                                # get the number of parts
                                num_of_parts = np.random.choice(np.arange(1, len(self.prob_for_num_of_parts)+1), p=self.prob_for_num_of_parts)
                        # get the sequence
                                audio, text_tokens = self.get_random_words_sequence_audio_tokens(num_of_words=self.num_of_words_in_sample, num_of_parts=num_of_parts)
                        else:
                                audio, text_tokens, _ = self.get_sequence_(index, num_of_words=self.num_of_words_in_sample)
                if self.augment:
                        # augment the audio
                        audio = self.augment_audio(audio)

                # compute log-Mel input features from input audio array
                input_features = processor.feature_extractor(audio, sampling_rate=SR).input_features[0]
                # compute input length of audio sample in seconds
                input_length = len(audio) / SR
                # processor.tokenizer.decode(text_tokens)
                return {"input_features": input_features, "input_length": input_length, "labels": text_tokens}

        def __len__(self):
                if self.is_eval_set:
                        return int(len(self.data)/self.num_of_words_in_sample)
                else:
                        if self.random:
                                # high number because of the augmentation
                                return 100000
                        else:
                                # The length is the (number of word in the data)/(number of words in sequance)
                                return len(self.data)

        def get_sequence_audio_text(self, sequence):
                audio = np.concatenate(sequence['audio'].values)
                text = " ".join(sequence['text'])
                audio_len = len(audio) / 16000
                text_tokens = processor.tokenizer.encode(text)
                text_len = len(text_tokens)
                return sequence, audio, text, audio_len, text_tokens, text_len
        
        def load_data(self,new_data , train, validation, test, nusachim=["ashkenazi"], load_cantillationless_data = False): 
                
                
                if load_cantillationless_data:
                        self.load_data_srt_mp3(train, validation, test)
                elif new_data:
                        if  (train==True and validation==False and test==False):
                                self.load_data_new(nusachim,train=True, validation=False, test=False)
                        elif (train==False and validation==True and test==False):
                                self.load_data_new(nusachim,train=False, validation=True, test=False)
                        elif (train==False and validation==False and test==True):
                                self.load_data_new(nusachim,train=False, validation=False, test=True)
                        else:
                                print(f"Invalid input. Please provide a valid input. train={train}, validation={validation}, test={test}")
                else:
                        self.load_data_old(validation)

        # methods for the new data
        def is_mp3_and_legal_length(self, filename, min_length=0.2, max_length=20):
                try:
                        audio = mutagen.mp3.MP3(filename)
                        if audio.info.length < min_length or audio.info.length > max_length:
                                return False
                        else:
                                return True
                except mutagen.MutagenError:
                        return False

        
        def is_text_with_nikud(self, text):
                for char in text:
                        if char in "ְֱֲֳִֵֶַָֹֺֻּֽ֑֖֛֢֣֤֥֦֧֪֚֭֮֒֓֔֕֗֘֙֜֝֞֟֠֡֨֩֫֬֯־ֿ׀ׁׂ׃ׅׄ׆ׇ": # string of all the nikud characters ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
                                return True
                return False

        def is_text_and_audio_pair_legal(self, text, filename):
                if not self.is_text_with_nikud(text):
                        return False
                if not self.is_mp3_and_legal_length(filename):
                        return False
                return True

        def load_data_new(self, nusachim, train, validation, test):
                # Load dataset.json
                if train:
                        file_path = 'train_data.json'
                elif validation:
                        file_path = 'validation_data.json'
                elif test:
                        file_path = 'test_data.json'
                else:
                        file_path = '03_dataset.json'

                with open(file_path, 'r', encoding='utf-8') as f:
                        predataset = json.load(f)
                
                audios = []
                texts = []
                for nusach in nusachim:
                        file_path = "dataset_" + nusach + ".npy"
                        if os.path.exists(file_path) and False: # we don't want to use the saved data as 1 file right now
                                data = np.load(file_path, allow_pickle=True).item()
                                audios.extend(data['audio'])
                                texts.extend(data['text'])
                        else:
                                if self.few_data:
                                        predataset[nusach] = predataset[nusach][:500]
                                        predataset['text'] = predataset['text'][:500]
                                        
                                missing_files = []
                                for index, audio_file in enumerate(tqdm(predataset[nusach], desc=f"Loading {nusach} nusach ({nusachim.index(nusach)+1}/{len(nusachim)})")):
                                        audio_path = os.path.join(audio_file)
                                        if self.is_text_and_audio_pair_legal(predataset['text'][index], audio_path):
                                                audio, sr = librosa.load(audio_path, sr=SR)
                                                audios.append(audio)
                                                texts.append(predataset['text'][index])
                                        else:
                                                missing_files.append((audio_path, predataset['text'][index], index))
                                # Save the missing files
                                with open('missing_files' + nusach + '.json', 'w', encoding='utf-8') as f:
                                        json.dump(missing_files, f, ensure_ascii=False, indent=4)
                                print("Num of missing files in " + nusach + " nusach: ", len(missing_files))
                                # Save the data for the next time
                                data = {"audio": audios, "text": texts}
                # create the dataset
                self.data = {"audio": audios, "text": texts}
                self.data = pd.DataFrame(self.data)
                

        # methods for the old data
        def prepare_transcript_str_to_list(self, text:str) -> list:
                """
                this function get a string of words and return a list of the words
                """
                return text.replace("    ", " ").replace("   ", " ").replace("  ", " ").replace(" ׀ ", "׀ ").replace(" ׀ ", "׀ ").replace("־", "־ ").replace("[1]", "").replace("\n", " ").replace("  ", " ").split(" ")

        def load_data_old(self, validation):
                success_count = 0
                fail_count = 0
                diff_list = []
                if validation:
                        transcript_folder = path + '/text_val'
                else:
                        transcript_folder = path + '/text'
                audio_folder = path + '/audio'
                timing_folder = path + '/time'
                audios = []
                text = []
                for filename in tqdm(os.listdir(transcript_folder)):
                        print("filename: ", filename)
                        if filename.endswith(".txt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.txt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                timing_path = os.path.join(timing_folder, filename)

                                audio, sr = librosa.load(audio_path, sr=16000)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = self.prepare_transcript_str_to_list(f.read())
                                with open(timing_path, 'r', encoding='utf-8') as f:
                                        timings = [float(time) for time in f.read().split(",")]
                                
                                if len(transcript) != len(timings):
                                        print("The length of the transcript and the timings is not equal")
                                        print("the difference (len(transcript) - len(timings)) is: ", len(transcript) - len(timings))
                                        diff_list.append((len(transcript) - len(timings), filename))
                                        fail_count += 1
                                else:
                                        success_count += 1
                                        for i, (word, start_time) in enumerate(zip(transcript, timings)):
                                                if i == len(transcript) - 1:
                                                        end_time = len(audio) / sr
                                                else:
                                                        end_time = timings[i+1]
                                                word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                                audios.append(word_audio)
                                                text.append(word)
                                print("success_count: ", success_count)
                                print("fail_count: ", fail_count)
                                
                
                # diff_hist:{1: 95, -1: 47, -6: 2, -2: 8, -5: 3, -3: 6, -7: 2, -4: 3, -8: 1, 3: 1, -9: 1, -26: 1}
                
                
                data_dict = {"audio": audios, "text": text}
                self.data = pd.DataFrame(data_dict)

        # For the new data without the cantillation
        def load_data_srt_mp3(self, train, validation, test):
                if train and not validation and not test:
                        folder = './train_cantillationless/'
                elif validation and not train and not test:
                        folder = './validation_cantillationless/'
                elif test and not train and not validation:
                        folder = './test_cantillationless/'
                else:
                        print("Invalid input. Please provide a valid input.")
                
                audio_folder = folder + 'audio'
                transcript_folder = folder + 'text' # the text is in srt format with times
                audios = []
                text = []
                import srt
                for filename in tqdm(os.listdir(transcript_folder)):
                        if filename.endswith(".srt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.srt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = list(srt.parse(f.read()))
                                audio, sr = librosa.load(audio_path, sr=16000)
                                for i, sub in enumerate(transcript):
                                        start_time = sub.start.total_seconds()
                                        end_time = sub.end.total_seconds()
                                        word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                        audios.append(word_audio)
                                        text.append(sub.content)
                data_dict = {"audio": audios, "text": text}
                self.data = pd.DataFrame(data_dict)
                

                        
        def get_data(self):
                return self.data

        def get_random_word(self):
                return random.choice(self.data)

        def get_sequence(self, start, end):
                return self.data[start:end]

        # the limit of whisper model
        # audio length of 30 seconds
        # text length of 448 tokens
        # I will take 20 words and check if the audio and text are in the limit
        def get_sequence_(self, start, num_of_words=20, random_cut_long=False):
                if start + num_of_words > len(self.data):
                        end = len(self.data)
                else:
                        end = start + num_of_words
                sequence = self.get_sequence(start, end)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens, end
                else: # cut the sequence
                        print("this sequence of ", num_of_words, " words is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))

                        if random_cut_long:
                                # divide into 2 parts and randomaly take one of them
                                if random.randint(0, 1) == 0:
                                        start = start + int(num_of_words/2)

                        if num_of_words>=2:
                                return self.get_sequence_(start, num_of_words=int(num_of_words/2), random_cut_long=random_cut_long)
                        else:
                                return self.get_sequence_(end, num_of_words=num_of_words, random_cut_long=random_cut_long)


        def get_dataset_slice_to_sequences(self, num_of_words):
                audios = []
                labels = []
                start = 0
                while start < len(self.data):
                        audio, label_feature, start = self.get_sequence_(start,num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        

        def get_random_sequence_(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence_(start)

        def get_random_sequence(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence(start, start+length)

        def get_random_words_sequence_audio_tokens(self, num_of_words, num_of_parts = None):
                """
                get sequence of random words (not logical sentences)
                createed from num_of_parts short sentences
                """
                if num_of_parts == None:
                        num_of_parts = num_of_words

                if num_of_parts > num_of_words:
                        print("num_of_parts can't be bigger than num_of_words")
                        print("so num_of_parts = num_of_words = ", num_of_words)
                        num_of_parts = num_of_words

                # num of words in each part
                num_of_words_in_parts = [num_of_words // num_of_parts + (1 if i < num_of_words % num_of_parts else 0) for i in range(num_of_parts)]

                sequence = {"audio": [], "text": []}
                for num_of_words_in_part in num_of_words_in_parts:
                        part = self.get_random_sequence(num_of_words_in_part)
                        sequence["audio"].extend(part["audio"])
                        sequence["text"].extend(part["text"])
                sequence = pd.DataFrame(sequence)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens
                else:
                        print("this sequence (of ", num_of_words, " words) is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))
                        return self.get_random_words_sequence_audio_tokens(num_of_words, num_of_parts)


        def get_dataset_slice_to_sequences_random_words(self, num_of_words, num_of_sequences=None, times = 5):
                audios = []
                labels = []
                if num_of_sequences:
                        num_of_sequences = num_of_sequences
                else:
                        num_of_sequences = int(len(self.data)*times/num_of_words)
                for i in range(num_of_sequences):
                        audio, label_feature = self.get_random_words_sequence_audio_tokens(num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        
        
        # methods for checking the data
        def get_longest_audio_index(self):
                """
                returns the index of longest audio in the dataset
                """
                index = np.argmax([len(audio) for audio in self.data['audio']])
                return index
        
        def get_longest_text_index(self):
                """
                returns the index of longest text in the dataset
                """
                index = np.argmax([len(text) for text in self.data['text']])
                return index
        
        def get_shortest_audio_index(self):
                """
                returns the index of shortest audio in the dataset
                """
                index = np.argmin([len(audio) for audio in self.data['audio']])
                return index
        
        def get_shortest_text_index(self):
                """
                returns the index of shortest text in the dataset
                """
                index = np.argmin([len(text) for text in self.data['text']])
                return index
        
        def check_the_data(self):
                """
                find the longest and shortest audio and text in the dataset
                and print and play them
                """
                index = self.get_longest_audio_index()
                print("longest audio index: ", index)
                print("longest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_audio_index()
                print("shortest audio index: ", index)
                print("shortest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_longest_text_index()
                print("longest text index: ", index)
                print("longest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_text_index()
                print("shortest text index: ", index)
                print("shortest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))


        def remove_word_by_index(self, index):
                """
                delete word from the dataset by index
                """
                if index < 0 or index >= len(self.data):
                        print("Invalid index. Please provide a valid index.")
                        return
                
                self.data.drop(index, inplace=True)
                self.data.reset_index(drop=True, inplace=True)

        def print_and_play_word_by_index(self,index):
                print(self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

        def augment_audio(self, audio):
                """
                augment the audio using audiomentations.
                each augmentation is done with random values.
                """
                augment = Compose([
                    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.03, p=0.5),
                    TimeStretch(min_rate=0.8, max_rate=3, p=1),
                    PitchShift(min_semitones=-8, max_semitones=8, p=1),
                    Shift(min_shift=0, max_shift=2, shift_unit="seconds", rollover=False),
                    RoomSimulator(),
                ])
                
                return audio

In [10]:
# !pip install audiomentations[extras]

In [11]:
if NEWDATA:
    train_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT)

else:
    train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=15, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2])

  0%|          | 0/361 [00:00<?, ?it/s]

filename:  Matot-6.txt


  0%|          | 1/361 [00:00<05:17,  1.13it/s]

success_count:  1
fail_count:  0
filename:  Beshalach-6.txt


  1%|          | 2/361 [00:01<03:45,  1.59it/s]

...

success_count:  164
fail_count:  137
filename:  Beshalach-2.txt
The length of the transcript and the timings is not equal
the difference (len(transcript) - len(timings)) is:  1
success_count:  164
fail_count:  138
filename:  KiTavo-7.txt


 84%|████████▍ | 303/361 [01:13<00:12,  4.83it/s]

The length of the transcript and the timings is not equal
the difference (len(transcript) - len(timings)) is:  1
success_count:  164
fail_count:  139
filename:  Vayishlach-5.txt


 84%|████████▍ | 304/361 [01:14<00:23,  2.43it/s]

success_count:  165
fail_count:  139
filename:  Behar-7.txt
The length of the transcript and the timings is not equal
the difference (len(transcript) - len(timings)) is:  1
success_count:  165
fail_count:  140
filename:  KiTavo-6.txt


In [ ]:
# # Load cantillationless data
# train_data = parashat_hashavua_dataset(few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, load_cantillationless_data=True)
# val_data = train_data

In [ ]:
a = train_data[0]

2024-05-14 18:40:18.322982: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 18:40:19.146803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [ ]:
if NEWDATA:
    val_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM)
else:
    val_data = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=15, random=RANDOM)
    

print(len(train_data))
print(len(val_data))

  0%|          | 0/4 [00:00<?, ?it/s]

filename:  AchreiMot-4.txt
success_count:  1
fail_count:  0
filename:  AchreiMot-1.txt


100%|██████████| 4/4 [00:00<00:00,  9.38it/s]

success_count:  2
fail_count:  0
filename:  AchreiMot-3.txt
The length of the transcript and the timings is not equal
the difference (len(transcript) - len(timings)) is:  1
success_count:  2
fail_count:  1
filename:  AchreiMot-2.txt
success_count:  3
fail_count:  1
100000
34


In [ ]:
path = "/home/user_7542/Desktop/project/cantillation/"

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import concurrent.futures
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he"




# # initialize the last layer of the model:
# model.proj_out.__init__(model.proj_out.in_features, len(processor.tokenizer))

# # add dropout
if DROPOUT:
    model.config.attention_dropout = DROPOUT
    model.config.dropout = DROPOUT
    model.config.activation_dropout = DROPOUT


if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS, 
    eval_steps=EVAL_STEPS,   
    logging_steps=25, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    run_name=RUN_NAME, # It doesn't work
)


In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback


class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [EvaluateFirstStepCallback()] if EVALUATE_FIRST_STEP else None
)



comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [ ]:
RUN_NAME

'medium_Random-True'

In [ ]:
flags_warnings()

trainer_state = trainer.train()

!!!NEWDATA==False!!!


Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
1,No log,5.510955,102.549020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


best f1 for Exact: 0
best pred: וידבר אדוני אל משה לימור, דבר אל אהרון ואל בניו, ואל כל בני ישראל, ואמרת
best label: וידב֥ר יהו֖ה אל־ מש֥ה לאמֽר׃ דב֨ר אֽל־ אהר֜ן ואל־ בנ֗יו ואל֙ כל־ בנ֣י ישרא֔ל ואמרת֖

worst f1 for Exact: 0
worst pred: וידבר אדוני אל משה לימור, דבר אל אהרון ואל בניו, ואל כל בני ישראל, ואמרת
worst label: וידב֥ר יהו֖ה אל־ מש֥ה לאמֽר׃ דב֨ר אֽל־ אהר֜ן ואל־ בנ֗יו ואל֙ כל־ בנ֣י ישרא֔ל ואמרת֖

worst f1 for Letter_Shift: 0
worst pred: וידבר אדוני אל משה לימור, דבר אל אהרון ואל בניו, ואל כל בני ישראל, ואמרת
worst label: וידב֥ר יהו֖ה אל־ מש֥ה לאמֽר׃ דב֨ר אֽל־ אהר֜ן ואל־ בנ֗יו ואל֙ כל־ בנ֣י ישרא֔ל ואמרת֖

worst f1 for Word_Level: 0
worst pred: וידבר אדוני אל משה לימור, דבר אל אהרון ואל בניו, ואל כל בני ישראל, ואמרת
worst label: וידב֥ר יהו֖ה אל־ מש֥ה לאמֽר׃ דב֨ר אֽל־ אהר֜ן ואל־ בנ֗יו ואל֙ כל־ בנ֣י ישרא֔ל ואמרת֖

worst f1 for Word_Shift: 0
worst pred: וידבר אדוני אל משה לימור, דבר אל אהרון ואל בניו, ואל כל בני ישראל, ואמרת
worst label: וידב֥ר יהו֖ה אל־ מש֥ה לאמֽר׃ דב֨ר אֽל־ אהר֜ן ו

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 10.50 MiB is free. Process 1945989 has 23.66 GiB memory in use. Of the allocated memory 22.59 GiB is allocated by PyTorch, and 810.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
kwargs = {
    # "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he-cantillation",
    "finetuned_from": BASE_MODEL_NAME,
    "tasks": "automatic-speech-recognition-cantillation",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

In [ ]:
trainer.lr_scheduler.get_lr()

In [ ]:
processor.tokenizer.decode(train_data[26]["labels"])

In [ ]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_avg_f1_Exact']
recall = history['eval_avg_recall_Exact']
precision = history['eval_avg_precision_Exact']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log_new.md")


In [ ]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




In [ ]:
# run a web server to see the tensorboard
!tensorboard --logdir ./whisper-medium-he-teamim-aviv-base --port 6006 --bind_all

# Test the model

In [ ]:
# load the model
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

# load the test data
test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, ra)

# create the data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# evaluate the model
results = trainer.evaluate() # we use evaluate to get the metrics
print(results)
# save the results to a json file
# create the results file
with open(f"results_{MODEL_NAME.split('/')[-1]}.json", 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
{'eval_loss': 0.7434155344963074, 'eval_wer': 12.154756685664182, 'eval_avg_precision_Exact': 0.9045080330977024, 'eval_avg_recall_Exact': 0.9046995984902677, 'eval_avg_f1_Exact': 0.9041808511811215, 'eval_avg_precision_Letter_Shift': 0.9262739837538204, 'eval_avg_recall_Letter_Shift': 0.9265417247288052, 'eval_avg_f1_Letter_Shift': 0.9259759561670436, 'eval_avg_precision_Word_Level': 0.9287635843313802, 'eval_avg_recall_Word_Level': 0.9291329677264785, 'eval_avg_f1_Word_Level': 0.928526987815817, 'eval_avg_precision_Word_Shift': 0.9744189603225837, 'eval_avg_recall_Word_Shift': 0.9756642194342305, 'eval_avg_f1_Word_Shift': 0.9746023104771994, 'eval_precision_median_exact': 1.0, 'eval_recall_median_exact': 1.0, 'eval_f1_median_exact': 1.0, 'eval_precision_max_exact': 1.0, 'eval_recall_max_exact': 1.0, 'eval_f1_max_exact': 1.0, 'eval_precision_min_Exact': 0.0, 'eval_recall_min_Exact': 0.0, 'eval_f1_min_Exact': 0.0, 'eval_precision_min_Letter_Shift': 0.0, 'eval_recall_min_Letter_Shift': 0.0, 'eval_f1_min_Letter_Shift': 0.0, 'eval_precision_min_Word_Level': 0.0, 'eval_recall_min_Word_Level': 0.0, 'eval_f1_min_Word_Level': 0.0, 'eval_precision_min_Word_Shift': 0.0, 'eval_recall_min_Word_Shift': 0.0, 'eval_f1_min_Word_Shift': 0.0, 'eval_runtime': 1534.2005, 'eval_samples_per_second': 1.755, 'eval_steps_per_second': 0.055}